# **Final Project for Numerical Analysis**

----

## *Analyzing efficacy of Gaussian Quadrature on Polynomial and Non-Polynomial Functions*

*Note: The book used for this class was "Numerical Analysis: The Mathematics of Scientific Computing, Kincaid and Cheney, 3rd edition". Any reference(s) to "the book" refer(s) to this book.*

Part One: Defining glaser_liu_rokhlin(), which we will refer to as $GLR()$, an algorithm which computes Gaussian Quadrature Formulas for function of $degree$ $n$ is $\Theta(n)$ time. 

While $GLR()$, based on a [paper](https://apps.dtic.mil/dtic/tr/fulltext/u2/a635869.pdf) by Andreas Glaser, Xiangtao Liu, and Vladimir Rokhlin, can work with various families of orthogonal polynomials, for our purposes we modified it to work specifically for Legendre polynomials, thereby allowing us to cut out some of the code used to generalize the algorithm "and make it work more efficiently for the Legendre polynomials.

In [2]:
using Polynomials, SpecialPolynomials, MTH229, Plots

function newton(p, x0)
    maxsteps = 25
    dp  = derivative(p)
    while maxsteps > 0
        px, dpx = p(x0),  dp(x0) #orthogonal_polyval_derivative(p, x0)
        Δ = px/dpx
        isnan(Δ) && return (x0, dpx)
        if isinf(Δ)
            x0 += sqrt(eps())  # give a nudge from minimum or maximum
            continue
        end
        x0 -= Δ
        min(abs(Δ),abs(px)) <= sqrt(eps())/100 && return  x0, dp(x0)# orthogonal_polyval_derivative(p, x0)[2]
        maxsteps -= 1
    end
    @warn "Newton's method did not converge from $x0"
    NaN
end
function RK(t0,  x0, F, h, n)
    k0 = h*F(t0,x0)
    local x1
    for i in 1:n
        t1 = t0 + h
        k1 = h * F(t0+h,x0+k0)
        x1 = x0 + 1/2 * (k0+k1)
        t0, x0, k0 = t1, x1, k1
    end
    x1
end

function prufer(π)
    n = Polynomials.degree(π)
    Π = typeof(π)
    (θ, x) -> begin
        dom = domain(Π)
        a,b,c,d, e = SpecialPolynomials.abcde(Π)
        p =  a*x^2 + b*x  +  c
        dp = 2a*x + b
        q = d*x +  e
        dq = d
        r  = -(a*n*(n-1)  + d*n)
        dr = 0
        -inv(sqrt(r/p) + (dr*p - dp*r + 2r*q)/(2r*p) * sin(2θ)/2)
    end
end


function glaser_liu_rokhlin_gauss_nodes(π, x0=0;
                                        m=10)
    symmetry = true
    n = Polynomials.degree(π)
    F = prufer(π)
    x = float(x0)

    # step 1 to get initial might be newton or might be RK
    if symmetry && iseven(n)
        # a max at π(x0)
        x = RK(pi/2, x0, F, -(pi/2)/m, m)
    end
    
    x1, dπ1 = newton(π, x)
    rts, dπrts = [x1], [dπ1]
    N = symmetry ? div(n+1,2) : n
    for i in 2:N
        xx = RK(pi/2, rts[end], F, -pi/m, m)
        x, dπx = newton(π, xx)
        push!(rts, x)
        push!(dπrts, dπx)
    end
    if symmetry
        if iseven(n)
            rts = vcat(-reverse(rts), rts)
            dπrts = vcat(-reverse(dπrts), dπrts)
        else
            rts = vcat(-reverse(rts[2:end]), rts)
            dπrts = vcat(-reverse(dπrts[2:end]), dπrts)            
        end
    end
    # get weights 
    weights = @. (1 * 2)/(1-(rts)^2)/((Polynomials.derivative(π)(rts))^2)
    rts,  weights
end

#a1, a2 = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, 6))
#@show a1, a2
#plot(a1, a2)
# x = variable()
# xs = range(-pi/2, pi/2, length = 200)
# ys =x.(xs)
# plot(xs, (ys))

glaser_liu_rokhlin_gauss_nodes (generic function with 2 methods)

### Demonstrating 7.3: Theorem 1
#### In Chapter 7.3 of the book, we see the following theorem:
*Let w be a positive weight frunction and let q be a nonzero polynomial of degree $n+1$ that is w-* **orthogonal** *to $\Pi_{n}$ in the sense that for any p in $\Pi_{n}$, we have</br>*  $\int\limits_a^b{q(x)p(x)w(x)dx}$ = 0
    *If* $x_{0},x_{1}, ... ,x_{n}$ *are the zeros of q, then the quadrature formula,* $\int\limits_a^b{f(x)w(x)dx}$ $\approx$ $\sum_{i=0}^{n}A_i f(x_i)$*, with coefficients given by*  $A_i = \int\limits_a^b{w(x) \Pi_{j=0}^{n}\frac{x-x_j}{x_i-x_j}dx}$ will be exact for all ${f ∈\Pi_{2n+1}}$ 

Here we demonstrate this theorem, the '$n$'s are shifted over however, so the quadrature formula will be "exact" up to but not including $2n$ (I put "exact" in quotes because techinically it is only as precise up to 10/20 decimal places etc.): 

In [21]:
n = 25
f(x) = x^n
xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, n))
for i in n:2n
    p = basis(Legendre, i)
    q = Polynomials.integrate(p, -1, 1)
    qq = sum(f(x)*w for (x,w) in zip(xs, ws))
    #@show "On iteration ", i, ", the integral of ", p, " from -1 to 1 is ", q, ", the approximated area is, ",  qq, "the approximation is accurate to ", q-qq
    @show i, q, qq, q-qq
end

#integrate(f(x), -1, 1)
# f(x) = exp(x)
# @show a = f(1)-f(-1)
# xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, 9))
# @show sum(f(x)*w for (x,w) in zip(xs, ws)) - a

(i, q, qq, q - qq) = (25, 0.0, 3.469446951953614e-18, -3.469446951953614e-18)
(i, q, qq, q - qq) = (26, 2.1502659097727417e-9, 3.469446951953614e-18, 2.1502659063032947e-9)
(i, q, qq, q - qq) = (27, 0.0, 3.469446951953614e-18, -3.469446951953614e-18)
(i, q, qq, q - qq) = (28, 3.7250698081692235e-9, 3.469446951953614e-18, 3.7250698046997766e-9)
(i, q, qq, q - qq) = (29, 0.0, 3.469446951953614e-18, -3.469446951953614e-18)
(i, q, qq, q - qq) = (30, 1.222224386143722e-7, 3.469446951953614e-18, 1.2222243861090276e-7)
(i, q, qq, q - qq) = (31, 0.0, 3.469446951953614e-18, -3.469446951953614e-18)
(i, q, qq, q - qq) = (32, -1.193120625631039e-7, 3.469446951953614e-18, -1.1931206256657334e-7)
(i, q, qq, q - qq) = (33, 0.0, 3.469446951953614e-18, -3.469446951953614e-18)
(i, q, qq, q - qq) = (34, 8.032657222711848e-8, 3.469446951953614e-18, 8.032657222364903e-8)
(i, q, qq, q - qq) = (35, 0.0, 3.469446951953614e-18, -3.469446951953614e-18)
(i, q, qq, q - qq) = (36, -3.581354394566194e-5, 3.46944695

In [33]:
f(x) = x^7+x^5+x^2
a = f(1)-f(-1)
n=7
xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, n))
for i in n:(2n)
    p = basis(Legendre, i)
    q = Polynomials.integrate(p, -1, 1)
    qq = sum(p(x)*w for (x,w) in zip(xs, ws))
    @show i, q - qq
end

(i, q - qq) = (7, 0.0)
(i, q - qq) = (8, -7.979727989493313e-17)
(i, q - qq) = (9, 0.0)
(i, q - qq) = (10, 1.2378986724570495e-14)
(i, q - qq) = (11, 0.0)
(i, q - qq) = (12, -3.0482560919864454e-14)
(i, q - qq) = (13, 0.0)
(i, q - qq) = (14, 0.4541175607603204)


### Efficacy of GLR() on Non-Polynomial Functions

$GLR()$ on $f(x)=e^x$. Notice how we get accuracy to 12 decimal places using only 6 points!

In [9]:
f(x) = exp(x)
MTH229.integrate(f, -1, 1)
@show a = f(1)-f(-1)
for i in 1:12
    xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, i))
    @show i, sum(f(x)*w for (x,w) in zip(xs, ws)) - a
end

a = f(1) - f(-1) = 2.3504023872876028
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1, -0.35040238728760276)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (2, -0.007706299377872483)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (3, -6.545860759210598e-5)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (4, -2.9513122612456755e-7)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (5, -8.247775795666712e-10)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (6, -1.568078999980571e-12)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (7, -1.7763568394002505e-15)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (8, 4.440892098500626e-16)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (9, 1.3322676295501878e-15)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (10, 0.0)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (11, 1.7763568394002505e-15)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (12, 8.881784197001252e-16)


Using a calculator, we can easily find $\int\limits_{-1}^1{cos(x)dx}$ $\approx 1.682941969615793 $. But we find that same result without integrating! And we used only 12 points! (okay, it's not that crazy, but still pretty cool)

In [14]:
f(x) = cos(x)
a = f(1)-f(-1)
for i in 1:12
    xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, i))
    q = sum(f(x)*w for (x,w) in zip(xs, ws))
    @show i, q, q - a
end

(i, q, q - a) = (1, 2.0, 2.0)
(i, q, q - a) = (2, 1.675823655389986, 1.675823655389986)
(i, q, q - a) = (3, 1.6830035477269165, 1.6830035477269165)
(i, q, q - a) = (4, 1.6829416886959734, 1.6829416886959734)
(i, q, q - a) = (5, 1.682941970407192, 1.682941970407192)
(i, q, q - a) = (6, 1.6829419696142796, 1.6829419696142796)
(i, q, q - a) = (7, 1.6829419696157952, 1.6829419696157952)
(i, q, q - a) = (8, 1.6829419696157928, 1.6829419696157928)
(i, q, q - a) = (9, 1.6829419696157937, 1.6829419696157937)
(i, q, q - a) = (10, 1.6829419696157928, 1.6829419696157928)
(i, q, q - a) = (11, 1.682941969615794, 1.682941969615794)
(i, q, q - a) = (12, 1.682941969615793, 1.682941969615793)


Using a calculator, we can easily find $\int\limits_{-1}^1{cos(x)^2+sin(x) dx}$ comes out to $\approx{1.454648713412841}$. But we find that same result without integrating! And we used only 8 points!

In [20]:
f(x) = cos(x)^2 + sin(x)
a = f(1)-f(-1)
for i in 1:15
    xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, i))
    q = sum(f(x)*w for (x,w) in zip(xs, ws))
    @show i, q, q - a
end

(i, q, q - a) = (1, 2.0, 0.317058030384207)
(i, q, q - a) = (2, 1.404192461982327, -0.27874950763346606)
(i, q, q - a) = (3, 1.4564451711321824, -0.2264967984836106)
(i, q, q - a) = (4, 1.4546153354356182, -0.22832663418017485)
(i, q, q - a) = (5, 1.454649094168245, -0.22829287544754795)
(i, q, q - a) = (6, 1.4546487104739119, -0.22829325914188114)
(i, q, q - a) = (7, 1.4546487134292159, -0.22829325618657714)
(i, q, q - a) = (8, 1.454648713412772, -0.2282932562030211)
(i, q, q - a) = (9, 1.4546487134128414, -0.2282932562029516)
(i, q, q - a) = (10, 1.4546487134128407, -0.22829325620295227)
(i, q, q - a) = (11, 1.4546487134128419, -0.22829325620295116)
(i, q, q - a) = (12, 1.4546487134128407, -0.22829325620295227)
(i, q, q - a) = (13, 1.4546487134128405, -0.2282932562029525)
(i, q, q - a) = (14, 1.45464871341284, -0.22829325620295293)
(i, q, q - a) = (15, 1.4546487134128407, -0.22829325620295227)


Now if we put some asymptotes at $x = -1, 1$, then we see that $GLR()$ using using a Legendre basis won't gut us anywhere, as the behavior of $Legendre(n)$ polynomials is such that as we get closer to $-1$ and $1$ we get more and more roots as n gets bigger, this makes it impossible to choose our points "wisely". Such is the case with the following function
Here we try it on a function $f(x)=\sqrt{1/(1-x^2)}$

In [3]:
f(x) = 1/ sqrt(1-x^2)
bound = 1/3+1/3+1/3.1
MTH229.integrate(f, -bound, bound)
@show a = f(bound)-f(-bound)
n = 1050
for i in n:n+5
    xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, i))
    @show i, sum(f(x)*w for (x,w) in zip(xs, ws)) - a
end

a = f(bound) - f(-bound) = 0.0
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1050, 3.139934985621325)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1051, 3.1399365621006363)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1052, 3.1399381355841363)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1053, 3.139939706080358)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1054, 3.1399412735979624)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1055, 3.13994283814556)
